In [2]:
import cv2
     
cc1 = cv2.imread('creditcard_digits1.jpg', 0)
cv2.imshow("Digits 1", cc1)
cv2.waitKey(0)
cc2 = cv2.imread('creditcard_digits2.jpg', 0)
cv2.imshow("Digits 2", cc2)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [4]:
cc1 = cv2.imread('creditcard_digits2.jpg', 0)
_, th2 = cv2.threshold(cc1, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
cv2.imshow("Digits 2 Thresholded", th2)
cv2.waitKey(0)
        
cv2.destroyAllWindows()

In [5]:
import os
 
def makedir(directory):
    """Creates a new directory if it does not exist"""
    if not os.path.exists(directory):
        os.makedirs(directory)
        return None, 0
    
for i in range(0,10):
    directory_name = "./credit_card/train/"+str(i)
    print(directory_name)
    makedir(directory_name) 
    
for i in range(0,10):
    directory_name = "./credit_card/test/"+str(i)
    print(directory_name)
    makedir(directory_name)

./credit_card/train/0
./credit_card/train/1
./credit_card/train/2
./credit_card/train/3
./credit_card/train/4
./credit_card/train/5
./credit_card/train/6
./credit_card/train/7
./credit_card/train/8
./credit_card/train/9
./credit_card/test/0
./credit_card/test/1
./credit_card/test/2
./credit_card/test/3
./credit_card/test/4
./credit_card/test/5
./credit_card/test/6
./credit_card/test/7
./credit_card/test/8
./credit_card/test/9


In [6]:
import cv2
import numpy as np 
import random
import cv2
from scipy.ndimage import convolve
     
def DigitAugmentation(frame, dim = 32):
    frame = cv2.resize(frame, None, fx=2, fy=2, interpolation = cv2.INTER_CUBIC)
    frame = cv2.cvtColor(frame, cv2.COLOR_GRAY2RGB)
    random_num = np.random.randint(0,9)
     
    if (random_num % 2 == 0):
        frame = add_noise(frame)
    if(random_num % 3 == 0):
        frame = pixelate(frame)
    if(random_num % 2 == 0):
        frame = stretch(frame)
    frame = cv2.resize(frame, (dim, dim), interpolation = cv2.INTER_AREA)
     
    return frame 
     
def add_noise(image):
    prob = random.uniform(0.01, 0.05)
    rnd = np.random.rand(image.shape[0], image.shape[1])
    noisy = image.copy()
    noisy[rnd < prob] = 0
    noisy[rnd > 1 - prob] = 1
    return noisy
     
def pixelate(image):
    dim = np.random.randint(8,12)
    image = cv2.resize(image, (dim, dim), interpolation = cv2.INTER_AREA)
    image = cv2.resize(image, (16, 16), interpolation = cv2.INTER_AREA)
    return image
     
def stretch(image):
        ran = np.random.randint(0,3)*2
        if np.random.randint(0,2) == 0:
            frame = cv2.resize(image, (32, ran+32), interpolation = cv2.INTER_AREA)
            return frame[int(ran/2):int(ran+32)-int(ran/2), 0:32]
        else:
            frame = cv2.resize(image, (ran+32, 32), interpolation = cv2.INTER_AREA)
            return frame[0:32, int(ran/2):int(ran+32)-int(ran/2)]
        
def pre_process(image, inv = False):
        try:
            gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        except:
            gray_image = image
            pass
        
        if inv == False:
            _, th2 = cv2.threshold(gray_image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
        else:
            _, th2 = cv2.threshold(gray_image, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
        resized = cv2.resize(th2, (32,32), interpolation = cv2.INTER_AREA)
        return resized

cc1 = cv2.imread('creditcard_digits2.jpg', 0)
_, th2 = cv2.threshold(cc1, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
cv2.imshow("cc1", th2)
cv2.waitKey(0)
cv2.destroyAllWindows()
     
region = [(0, 0), (35, 48)]
     
# Assigns values to each region for ease of interpretation
top_left_y = region[0][1]
bottom_right_y = region[1][1]
top_left_x = region[0][0]
bottom_right_x = region[1][0]
     
for i in range(0,1): #We only look at the first digit in testing out augmentation functions
    roi = cc1[top_left_y:bottom_right_y, top_left_x:bottom_right_x]
    for j in range(0,10):
            roi2 = DigitAugmentation(roi)
            roi_otsu = pre_process(roi2, inv = False)
            cv2.imshow("otsu", roi_otsu)
            cv2.waitKey(0)
            
    cv2.destroyAllWindows()

In [14]:
cc1 = cv2.imread('creditcard_digits1.jpg', 0)
 
_, th2 = cv2.threshold(cc1, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
cv2.imshow("cc1", th2)
cv2.imshow("creditcard_digits1", cc1)
cv2.waitKey(0)
cv2.destroyAllWindows()
 
region = [(2, 19), (50, 72)]
 
top_left_y = region[0][1]
bottom_right_y = region[1][1]
top_left_x = region[0][0]
bottom_right_x = region[1][0]
 
for i in range(0,10):   
    # We jump the next digit each time we loop
    if i > 0:
        top_left_x = top_left_x + 59
        bottom_right_x = bottom_right_x + 59
 
    roi = cc1[top_left_y:bottom_right_y, top_left_x:bottom_right_x]
    print("Augmenting Digit - ", str(i))
    # We create 200 versions of each image for our dataset
    for j in range(0,2000):
        roi2 = DigitAugmentation(roi)
        roi_otsu = pre_process(roi2, inv = True)
        cv2.imwrite("/home/saksham_nuclear_pc/IEEE Mega Project 3.0/credit_card/train/"+str(i)+"/_1_"+str(j)+".jpg", roi_otsu)
        
cv2.destroyAllWindows()

Augmenting Digit -  0
Augmenting Digit -  1
Augmenting Digit -  2
Augmenting Digit -  3
Augmenting Digit -  4
Augmenting Digit -  5
Augmenting Digit -  6
Augmenting Digit -  7
Augmenting Digit -  8
Augmenting Digit -  9


In [15]:
cc1 = cv2.imread('creditcard_digits2.jpg', 0)
_, th2 = cv2.threshold(cc1, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
cv2.imshow("cc1", th2)
cv2.waitKey(0)
cv2.destroyAllWindows()
 
region = [(0, 0), (35, 48)]
 
top_left_y = region[0][1]
bottom_right_y = region[1][1]
top_left_x = region[0][0]
bottom_right_x = region[1][0]
 
for i in range(0,10):   
    if i > 0:
        # We jump the next digit each time we loop
        top_left_x = top_left_x + 35
        bottom_right_x = bottom_right_x + 35
 
    roi = cc1[top_left_y:bottom_right_y, top_left_x:bottom_right_x]
    print("Augmenting Digit - ", str(i))
    # We create 200 versions of each image for our dataset
    for j in range(0,2000):
        roi2 = DigitAugmentation(roi)
        roi_otsu = pre_process(roi2, inv = False)
        cv2.imwrite("/home/saksham_nuclear_pc/IEEE Mega Project 3.0/credit_card/train/"+str(i)+"/_2_"+str(j)+".jpg", roi_otsu)

cv2.destroyAllWindows()

Augmenting Digit -  0
Augmenting Digit -  1
Augmenting Digit -  2
Augmenting Digit -  3
Augmenting Digit -  4
Augmenting Digit -  5
Augmenting Digit -  6
Augmenting Digit -  7
Augmenting Digit -  8
Augmenting Digit -  9


In [7]:
cc1 = cv2.imread('creditcard_digits1.jpg', 0)
     
_, th2 = cv2.threshold(cc1, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
cv2.imshow("cc1", th2)
cv2.imshow("creditcard_digits1", cc1)
cv2.waitKey(0)
cv2.destroyAllWindows()
     
region = [(2, 19), (50, 72)]
     
top_left_y = region[0][1]
bottom_right_y = region[1][1]
top_left_x = region[0][0]
bottom_right_x = region[1][0]
     
for i in range(0,10):   
    if i > 0:
        top_left_x = top_left_x + 59
        bottom_right_x = bottom_right_x + 59
     
    roi = cc1[top_left_y:bottom_right_y, top_left_x:bottom_right_x]
    print("Augmenting Digit - ", str(i))
    for j in range(0,200):
        roi2 = DigitAugmentation(roi)
        roi_otsu = pre_process(roi2, inv = True)
        cv2.imwrite("/home/saksham_nuclear_pc/IEEE Mega Project 3.0/credit_card/test/"+str(i)+"/_1_"+str(j)+".jpg", roi_otsu)

cc1 = cv2.imread('creditcard_digits2.jpg', 0)
_, th2 = cv2.threshold(cc1, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
cv2.imshow("cc1", th2)
cv2.waitKey(0)
cv2.destroyAllWindows()
     
region = [(0, 0), (35, 48)]
     
top_left_y = region[0][1]
bottom_right_y = region[1][1]
top_left_x = region[0][0]
bottom_right_x = region[1][0]
     
for i in range(0,10):   
    if i > 0:
        top_left_x = top_left_x + 35
        bottom_right_x = bottom_right_x + 35
     
    roi = cc1[top_left_y:bottom_right_y, top_left_x:bottom_right_x]
    print("Augmenting Digit - ", str(i))
    for j in range(0,200):
        roi2 = DigitAugmentation(roi)
        roi_otsu = pre_process(roi2, inv = False)
        cv2.imwrite("/home/saksham_nuclear_pc/IEEE Mega Project 3.0/credit_card/test/"+str(i)+"/_2_"+str(j)+".jpg", roi_otsu)

Augmenting Digit -  0
Augmenting Digit -  1
Augmenting Digit -  2
Augmenting Digit -  3
Augmenting Digit -  4
Augmenting Digit -  5
Augmenting Digit -  6
Augmenting Digit -  7
Augmenting Digit -  8
Augmenting Digit -  9
Augmenting Digit -  0
Augmenting Digit -  1
Augmenting Digit -  2
Augmenting Digit -  3
Augmenting Digit -  4
Augmenting Digit -  5
Augmenting Digit -  6
Augmenting Digit -  7
Augmenting Digit -  8
Augmenting Digit -  9
